In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Đọc dữ liệu từ file CSV
def read_csv_data(file_path):
    df = pd.read_csv(file_path, sep="\t")  # Tách cột bằng dấu tab nếu cần
    
    # Xác định kho (hàng đầu tiên)
    depot = df.iloc[0]
    
    # Xác định danh sách khách hàng
    customers = df.iloc[1:].reset_index(drop=True)
    
    return depot, customers

# Hiển thị dữ liệu
file_path = "F:/KLTN/100_Customer/h100c101.csv"  # Đặt đường dẫn file

depot, customers = read_csv_data(file_path)
print("📌 Kho hàng:", depot)
print("📌 Số lượng khách hàng:", len(customers))
print(customers.head())
print(customers.columns)


📌 Kho hàng: x,y,demand,open,close,servicetime,time    33.33333333,41.66666667,0,0,1236,5,0
Name: 0, dtype: object
📌 Số lượng khách hàng: 100
  x,y,demand,open,close,servicetime,time
0        37.5,58.33333333,30,825,870,5,0
1            35,54.16666667,10,15,67,5,0
2          18.33333333,62.5,30,30,92,5,0
3            25,41.66666667,10,10,73,5,0
4  8.333333333,33.33333333,30,31,100,5,0
Index(['x,y,demand,open,close,servicetime,time'], dtype='object')


In [24]:
import pandas as pd
import numpy as np

file_path = "F:/KLTN/100_Customer/h100c101.csv"

# Đọc file CSV
df = pd.read_csv(file_path,delimiter=",", header=0)

# Kiểm tra số cột thực tế
print("Số lượng cột thực tế:", df.shape[1])
print("Dữ liệu mẫu:\n", df.head())
print()

# Xác định kho hàng (hàng đầu tiên sau tiêu đề)
depot = df.iloc[0]
print("Kho hàng:\n", depot)
print()

customers = df.iloc[1:].reset_index(drop=True)

# Chuyen doi toa do thanh numpy array
depot_location = np.array(depot[['x', 'y']].values) # Kho hang
customer_locations = customers[['x', 'y']].values # Khach hang

print("Tọa độ kho hàng:", depot_location)
print("Tọa độ khách hàng:\n", customer_locations)

# Trọng lượng hàng và giới hạn xe
demands = customers['demand'].values
truck_capacity = 1300  # Định nghĩa tải trọng xe

# Giới hạn thời gian phục vụ
time_windows = customers[['open', 'close']].values
service_times = customers['servicetime'].values
arrival_times = customers['time'].values

# Vận tốc xe (km/phút)
truck_speed = 50 / 60.0

# Hiển thị thông tin cơ bản
print(f"Number of Customers: {len(customers)}")
print(f"Truck Capacity: {truck_capacity}")
print(f"Speed of Trucks: {truck_speed} km/m")


Số lượng cột thực tế: 7
Dữ liệu mẫu:
            x          y  demand  open  close  servicetime  time
0  33.333333  41.666667       0     0   1236            5   0.0
1  37.500000  58.333333      30   825    870            5   0.0
2  35.000000  54.166667      10    15     67            5   0.0
3  18.333333  62.500000      30    30     92            5   0.0
4  25.000000  41.666667      10    10     73            5   0.0

Kho hàng:
 x                33.333333
y                41.666667
demand            0.000000
open              0.000000
close          1236.000000
servicetime       5.000000
time              0.000000
Name: 0, dtype: float64

Tọa độ kho hàng: [33.33333333 41.66666667]
Tọa độ khách hàng:
 [[37.5        58.33333333]
 [35.         54.16666667]
 [18.33333333 62.5       ]
 [25.         41.66666667]
 [ 8.33333333 33.33333333]
 [ 0.         37.5       ]
 [27.5        26.66666667]
 [27.5        29.16666667]
 [23.33333333 25.        ]
 [33.33333333 12.5       ]
 [41.66666667 25.  

In [25]:
import numpy as np

class Vehicle:
    def __init__(self, vehicle_id, capacity, speed, depot_location):
        self.vehicle_id = vehicle_id
        self.capacity = capacity  # Sức chứa tối đa
        self.speed = speed  # Vận tốc di chuyển (km/phút)
        self.current_location = tuple(depot_location)  # Vị trí hiện tại (bắt đầu tại kho)
        self.route = [self.current_location]  # Lộ trình xe (bắt đầu từ kho)
        self.load = 0  # Khối lượng hàng đang chở
        self.time = 0  # Thời gian hiện tại của xe
    
    def can_serve(self, demand, time_window):
        """ Kiểm tra xem xe có thể phục vụ khách hàng không """
        return (self.load + demand <= self.capacity) and (self.time <= time_window[1])
    
    def serve_customer(self, customer_index, customer_locations, demands, service_times):
        """ Phục vụ khách hàng và cập nhật trạng thái xe """
        customer_x, customer_y = tuple(customer_locations[customer_index])
        distance = np.linalg.norm(np.array([customer_x, customer_y]) - np.array(self.current_location))
        travel_time = distance / self.speed  # Thời gian di chuyển
        
        # Cập nhật trạng thái xe
        self.current_location = (customer_x, customer_y)
        self.route.append(self.current_location)

        # Kiểm tra tải trọng trước khi cập nhật
        if self.load + demands[customer_index] <= self.capacity:
            self.load += demands[customer_index]
        
        self.time += travel_time + service_times[customer_index]  # Cộng thời gian phục vụ


class Environment:
    def __init__(self, num_vehicles, truck_capacity, truck_speed, depot_location, customer_locations, demands, time_windows, service_times):
        self.vehicles = [Vehicle(i, truck_capacity, truck_speed, depot_location) for i in range(num_vehicles)]
        self.unvisited_customers = list(range(len(customer_locations)))  # Danh sách khách hàng chưa phục vụ
        self.customer_locations = customer_locations
        self.demands = demands
        self.time_windows = time_windows
        self.service_times = service_times
    
    def assign_vehicle(self):
        """ Phân công xe tải dựa trên xe gần nhất có thể phục vụ """
        while self.unvisited_customers:
            for vehicle in self.vehicles:
                if not self.unvisited_customers:
                    break  # Thoát nếu không còn khách hàng

                best_customer = None
                min_distance = float('inf')

                for customer_index in self.unvisited_customers:
                    if vehicle.can_serve(self.demands[customer_index], self.time_windows[customer_index]):
                        # Tính khoảng cách đến khách hàng
                        customer_x, customer_y = tuple(self.customer_locations[customer_index])
                        distance = np.linalg.norm(np.array([customer_x, customer_y]) - np.array(vehicle.current_location))

                        # Chọn khách hàng gần nhất
                        if distance < min_distance:
                            min_distance = distance
                            best_customer = customer_index

                if best_customer is not None:
                    vehicle.serve_customer(best_customer, self.customer_locations, self.demands, self.service_times)
                    self.unvisited_customers.remove(best_customer)

    def get_routes(self):
        """ Lấy tuyến đường của từng xe """
        return [vehicle.route for vehicle in self.vehicles]
